# 1. Introduction

## 1.1 Imports
Import libraries here.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from kuberspatiotemporal import CompoundModel, Feature, SpatialModel, KuberModel
from kuberspatiotemporal.tools import make_ellipses

In [ ]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
import matplotlib.pyplot as plt

# 2. Load Data

In [ ]:
data = pd.read_json('data/spatial_data_lisboa.json')

In [ ]:
data.head(2)

In [ ]:
# Filter office and home locations
data = data[['latitude', 'longitude', 'timestamp']]
data = data[(
    (((data['latitude']>38.710104) & (data['latitude']<38.710648)) & ((data['longitude']>-9.139794) & (data['longitude']<-9.1389)))
 | (((data['latitude']>38.725828) & (data['latitude']<38.726512)) & ((data['longitude']>-9.133944) & (data['longitude']<-9.133169))))]
data['time'] = [ts.hour + ts.minute/60 + ts.second/3600 for ts in data.timestamp]
data['weekday'] = [ts.dayofweek for ts in data.timestamp]

In [ ]:
display(data, data.hist(column="weekday", bins=6))

# 3. Learn Spatiotemporal Model - 4D

In [ ]:
limits_ = [np.min(data[['latitude', 'longitude', 'time']].values, axis=0),np.max(data[['latitude', 'longitude', 'time']].values, axis=0)]

In [ ]:
kst = CompoundModel(
    n_dim=4,
    n_iterations=200,
    scaling_parameter=1.1,
    nonparametric=True,
    online_learning=False,
    loa=True,
    features=[
        Feature(SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits_), [0, 1, 2]),
        Feature(KuberModel(n_symbols=7), [3])
    ],
)

In [ ]:
pipeline = make_pipeline(
    make_column_transformer(
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "latitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "longitude"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "time"),
        (FunctionTransformer(lambda x: np.array(x).reshape(-1, 1)), "weekday"),
    ),
    kst,
)

In [ ]:
pipeline.fit(data[['latitude', 'longitude', 'time', 'weekday']])

In [ ]:
kst.score_threshold = kst.get_score_threshold(data[['latitude', 'longitude', 'time', 'weekday']].values, quantile=0.005)
kst.score_threshold

In [ ]:
kst.score_threshold = 0.8

In [ ]:
kst

In [ ]:
kst.loa=False

In [ ]:
# home
# 38.7263483,-9.135931
# office
# 38.7104174,-9.1417113
kst.score(np.array([[38.7263483,-9.135931, 21,0]]))

In [ ]:
pipeline.score(pd.DataFrame([[38.7104174,-9.1417113,16,2]], columns=['latitude', 'longitude', 'time', 'weekday']))

## 4.1 Spatio Model

In [ ]:
limits = np.array([[data.latitude.min(), data.longitude.min()],[data.latitude.max(), data.longitude.max()]])
limits = limits + np.array([[-0.01], [0.01]])

In [ ]:
spatio_model = SpatialModel(n_dim=2, min_eigval=0.000000001, nonparametric=True, n_iterations=200, limits=limits,
                    scaling_parameter=100, loa=True, decay=5)

In [ ]:
spatio_model.fit(data[['latitude', 'longitude']].values)

In [ ]:
spatio_model.box = 0.05

In [ ]:
#office
spatio_score = spatio_model.score(np.array([[38.7104174,-9.1417113]]))
spatio_score

## 4.2 Spatiotemporal Model

In [ ]:
spatiotemporal_model = SpatialModel(n_dim=3, min_eigval=1e-9, limits=limits_, n_iterations=200, 
                                    scaling_parameter=1.1, nonparametric=True, online_learning=False,
                                    loa=True)

In [ ]:
spatiotemporal_model.fit(data[['latitude', 'longitude', 'time']].values)

In [ ]:
spatiotemporal_model.box = [0.05, 0.05, 1]

In [ ]:
#office
spatiotemporal_score = spatiotemporal_model.score(np.array([[38.7104174, -9.1417113, 12]]))
spatiotemporal_score

## 4.3 Weekday Model

In [ ]:
weekday_model = KuberModel(n_symbols=7, n_components=7, loa=True)

In [ ]:
weekday_model.fit(data[['weekday']].values)

In [ ]:
weekday_model.box = None

In [ ]:
weekday_score = weekday_model.score(np.array([[2]]))
weekday_score

In [ ]:
weekday_score*spatiotemporal_score